In [23]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import ipywidgets as widgets
import itertools
from IPython.display import display

plt.rcParams['figure.figsize'] = [14, 8]

In [38]:
base_path = "results/"
folders= ["share", "no_share"]
tests= [
#     "multiplexer", 
#     "parity", 
#     "regression", 
    "old_regression", 
#     "older_regression", 
#     "multiplexer_n1000", 
#     "parity_n1000",
#     "regression_n1000"
    "tmp"
]
# tests= ["multiplexer_n1000", "parity_n1000", "regression_n1000"]
# runs = ["1", "2", "3", "4", "5"]
runs = ["1"]

col_names = []

dfs = {}
for t in tests: 
    dfs[t] = {}
    for f in folders:
        dfs[t][f] = []
        for r in runs:
#             file_name = base_path + f + "/diversity_" + t + "_"+ r + "_n1000.csv"
            file_name = base_path + f + "/diversity_" + t + "_" + r  + ".csv"
            df = pd.read_csv(file_name, sep=';')
            dfs[t][f].append(df)
            col_names = df.columns[1:]
            
dfs[t][f][0].head(2)

,gen,fitness,n_species,p_isomorphisms,unique_fitness,entropy,total_gen_markers,active_gen_markers,inactive_gen_markers,avg_delta,std_delta
0,0,265.274339,100,84,78,6.146112,6260,806,5772,0.832106,0.029323
1,1,265.274339,69,91,91,6.441210,6177,958,5557,0.819014,0.063549


In [39]:
def normalize_column(df, c_name):
    c = df[c_name]
    df[c_name] = (c-c.min())/(c.max()-c.min())

In [40]:
def plot_graphs(dfs, test_name, gen_range, cols):
    for k, v in dfs[test_name].items():
        host = True
        for column in cols:
            full_data = [d[column] for d in v]
            
            data = []
            for d in itertools.zip_longest(*full_data, fillvalue = 0):
                data.append(d)
                
            start = min(gen_range[0], len(data))
            finish = min(gen_range[1], len(data))
            steps = [i for i in range(start, finish)]
            
            data = data[start:finish]
            avg_arr = [sum(d)/len(d) for d in data]
            min_arr = [min(d) for d in data]
            max_arr = [max(d) for d in data]

            plt.plot(steps, avg_arr, lw =1.3, label=f"{k} {column}")
            plt.fill_between(steps, min_arr,max_arr, alpha = 0.15)

    plt.legend(loc="upper left")
    plt.show()

In [41]:
test = widgets.ToggleButtons(
    options=tests,
    description='Test:',
    disabled=False,
    button_style='',
)

gen_range = widgets.IntRangeSlider(
    value=[10, 1000],
    min=10,
    max=1000,
    step=10,
    description='Generations:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

parameters = []
for col in col_names:
    cb = widgets.Checkbox(
        value=False,
        description=col,
        disabled=False,
        indent=False
    )
    parameters.append(cb)
    
button = widgets.Button(description="Render Graph")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        cols = [p.description for p in parameters if p.value]
        if len(cols) > 0:
            plot_graphs(dfs, test.value, gen_range.value, cols)

button.on_click(on_button_clicked)

In [42]:
display(test)
display(gen_range)

half_len = len(parameters)//2
input_widgets_1 = widgets.HBox(parameters[:half_len])
display(input_widgets_1)
input_widgets_2 = widgets.HBox(parameters[half_len:])
display(input_widgets_2)

display(button, output)

ToggleButtons(description='Test:', options=('old_regression', 'tmp'), value='old_regression')

IntRangeSlider(value=(10, 1000), continuous_update=False, description='Generations:', max=1000, min=10, step=1…

Button(description='Render Graph', style=ButtonStyle())

Output()